<a href="https://colab.research.google.com/github/mamadoz79/NLP-CA/blob/main/machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional
import numpy as np
import string
import random
import re

In [2]:
batch_size = 256
epochs = 10
latent_dim = 256

In [3]:
en = open("/content/drive/MyDrive/NLP/TEP.en-fa.en", encoding='utf-8')
fa = open("/content/drive/MyDrive/NLP/TEP.en-fa.fa", encoding='utf-8')

In [4]:
strp = lambda seq : re.sub(" +", " ", ' '.join(word.strip(string.punctuation) for word in seq.split()))

In [5]:
lines = [[strp(e[:-2]), strp(f[:-2])] for (e, f) in zip(en, fa)]

In [6]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
for line in lines[:30000]:
    input_text, target_text = line
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [7]:
del lines

In [8]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [9]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 43
Number of unique output tokens: 86
Max sequence length for inputs: 154
Max sequence length for outputs: 140


In [10]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [11]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [12]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [13]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [14]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [15]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])

In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 43)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 86)]   0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        307200      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                              

In [ ]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,batch_size=64, epochs=100)

Epoch 1/100
157/157 [==============================] - 5s 30ms/step - loss: 0.4667 - accuracy: 0.8656
Epoch 2/100
157/157 [==============================] - 5s 29ms/step - loss: 0.4565 - accuracy: 0.8684
Epoch 3/100
157/157 [==============================] - 5s 29ms/step - loss: 0.4475 - accuracy: 0.8709
Epoch 4/100
157/157 [==============================] - 5s 29ms/step - loss: 0.4389 - accuracy: 0.8733
Epoch 5/100
157/157 [==============================] - 5s 29ms/step - loss: 0.4316 - accuracy: 0.8753
Epoch 6/100
157/157 [==============================] - 5s 29ms/step - loss: 0.4243 - accuracy: 0.8775
Epoch 7/100
157/157 [==============================] - 5s 29ms/step - loss: 0.4177 - accuracy: 0.8792
Epoch 8/100
157/157 [==============================] - 5s 29ms/step - loss: 0.4116 - accuracy: 0.8808
Epoch 9/100
157/157 [==============================] - 5s 29ms/step - loss: 0.4058 - accuracy: 0.8825
Epoch 10/100
157/157 [==============================] - 5s 29ms/step - loss: 0.400

In [18]:
model.save('MODEL2.h5')

In [19]:
!cp -r "/content/MODEL2.h5" "/content/drive/MyDrive/NLP/MODEL2.h5"

In [20]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)

In [21]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [22]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        states_value = [h, c]
    return decoded_sentence